In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
def correct_anomalous_values(value):
    # Verificar se o valor é uma string que representa um número
    if isinstance(value, str):
        try:
            value = float(value)
        except ValueError:
            return np.nan  # Se a conversão falhar, retornar NaN

    # Verificar se o valor é um número
    if isinstance(value, (int, float)):
        if value > 100:  # Valor suspeito
            return value / 1000  # Corrigir dividindo por 1000
        return value  # Valor normal

    return np.nan  # Para outros casos, retornar NaN

In [ ]:
data = pd.read_csv('GlobalLandTemperaturesByCountryV2.csv')
data['dt'] = pd.to_datetime(data['dt'])

data_cleaned = data.dropna(subset=['AverageTemperature'])
data_cleaned['AverageTemperature'] = data_cleaned['AverageTemperature'].apply(correct_anomalous_values)
data_cleaned = data_cleaned.dropna(subset=['AverageTemperature'])
data_cleaned['Year'] = data_cleaned['dt'].dt.year
data_cleaned['Month'] = data_cleaned['dt'].dt.month


monthly_avg_temp = data_cleaned.groupby(['Country','Year', 'Month'])['AverageTemperature'].mean().reset_index()

<ipython-input-133-4fc16cbe6907>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['AverageTemperature'] = data_cleaned['AverageTemperature'].apply(correct_anomalous_values)


In [ ]:
p = range(0,3) # Ordem da parte auto-regressiva
d = range(0,2) # Ordem de diferenciação
q = range(0,3) # Ordem da média móvel
parameters = {'order': [(x, y, z) for x in p for y in d for z in q]}

def get_country_info(country_name, prediction_date):
  country_data = monthly_avg_temp[monthly_avg_temp['Country'] == country_name]

  train_data = country_data[country_data['Year'] <= 2000]['AverageTemperature']
  test_data = country_data[((country_data['Year'] > 2000) & (country_data['Year'] < 2013)) | ((country_data['Year'] == 2013) & (country_data['Month'] <= 9))]['AverageTemperature']

  best_mse = float('inf')
  best_order = None
  mae = None
  for order in parameters['order']:
    try:
      model = ARIMA(train_data, order=order)
      model_fit = model.fit()

      forecast = model_fit.forecast(steps=len(test_data))
      mse = mean_squared_error(test_data, forecast)

      if mse < best_mse:
        mae = mean_absolute_error(test_data, forecast)
        best_mse = mse
        best_order = order
    except Exception as e:
      #print(e)
      continue

  final_model = ARIMA(country_data['AverageTemperature'], order=best_order)
  final_model_fit = final_model.fit()

  months_diff = (pd.to_datetime(prediction_date).year - 2013) * 12 + pd.to_datetime(prediction_date).month - 9
  forecast_all = final_model_fit.forecast(steps=months_diff)
  results[country_name] = {
      'test_data': test_data,
      'rmse': np.sqrt(best_mse),
      'mae': mae,
      'best_order': best_order,
      'forecast': forecast_all
  }


In [ ]:
prediction_date = "2024-03-01"
get_country_info("Canada", prediction_date)

for country in results:
    print(f"Country: {country}")
    print(f"Mean Squared Error: {results[country]['rmse']}")
    print(f"MAE: {results[country]['mae']}")
    print("Forecast:")
    print(len(results[country]['forecast']))
    print(results[country]['forecast'].iloc[-1])
    print("\n")

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this

Country: Brazil
Mean Squared Error: 0.8638543712058718
MAE: 0.6664666221792378
Forecast:
126
24.850773680817976


Country: Canada
Mean Squared Error: 2.990511660699722
MAE: 2.528852438187538
Forecast:
126
-13.178973500906004




/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
